## 👩‍🍳 The Secret Sauce to Speckle Data Connections
The Python SDK can be found here: https://github.com/specklesystems/specklepy, with docs here: https://speckle.guide/dev/python.html

#### 📇 Step 01 - Define project variables

In [ ]:
HOST = "https://app.speckle.systems/"
AUTHENTICATION_TOKEN = ""

#### 📚 Step 02 - Checking availability of required libraries and install (if req.)

In [ ]:
import importlib
import subprocess
import sys

additional_libraries = ["specklepy", "ipywidgets", "matplotlib", "seaborn", "plotly"]

for library in additional_libraries:
    try:
        # Try importing the library
        importlib.import_module(library)
    except ImportError:
        print(f"{library} not found. Installing...")
        # Install the library using subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", library])
        # Retry import after installation
        globals()[library] = importlib.import_module(library)

#### 👔 Step 03 - Introduce the SpeckleClient

In [ ]:
from specklepy.api.client import SpeckleClient
from specklepy.core.api import operations
from specklepy.transports.server import ServerTransport

client = SpeckleClient(host=HOST)
client.authenticate_with_token(token=AUTHENTICATION_TOKEN)

#### 🧐 Step 04 - Fancy Widgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Fetch the list of streams
streams = client.stream.list()
stream_options = {stream.name: stream.id for stream in streams}

# Create a dropdown for streams
stream_dropdown = widgets.Dropdown(
    options=stream_options,
    description='Stream:',
    value=list(stream_options.values())[0]  # Default to the first stream
)

# Label widget to display the selected stream ID
stream_id_label = widgets.Label(value=f"Selected Stream ID: {stream_dropdown.value}")

# Define a function to update branch dropdown and stream ID label based on selected stream
def update_branch_options(change):
    selected_stream_id = change['new']
    branches = client.branch.list(selected_stream_id)
    branch_options = {branch.name: branch.name for branch in branches}
    branch_dropdown.options = branch_options
    
    # Update the stream ID label
    stream_id_label.value = f"Selected Stream ID: {selected_stream_id}"

# Fetch branches for the initially selected stream
initial_branches = client.branch.list(stream_dropdown.value)
branch_options = {branch.name: branch.name for branch in initial_branches}

# Create a dropdown for branches
branch_dropdown = widgets.Dropdown(
    options=branch_options,
    description='Branch:',
    value=list(branch_options.values())[0]  # Default to the first branch
)

# Link the stream dropdown to update branches and the stream ID label when a new stream is selected
stream_dropdown.observe(update_branch_options, names='value')

# Display the dropdowns and the stream ID label
display(stream_dropdown, stream_id_label, branch_dropdown)

#### ⚙️ Step 05 - Use widget outputs

In [ ]:
selected_stream_id = stream_dropdown.value
selected_branch_name = branch_dropdown.value
transport = ServerTransport(selected_stream_id, client) 

In [ ]:
branch = client.branch.get(stream_id=selected_stream_id, name=selected_branch_name)
if branch.commits.items:
    model_data = operations.receive(branch.commits.items[0].referencedObject, transport)
    model_data.get_member_names()
else:
    print("Model has no commits.")